In [3]:
import requests
from urllib.parse import urlencode
from bs4 import BeautifulSoup
import datetime
from datetime import datetime
import time
import json
import requests
from requests.exceptions import SSLError
import psycopg2
import random
import csv
from email.mime.text import MIMEText
from email.utils import COMMASPACE
import smtplib

headers = {
    'authority': 'moto.av.by',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'ru,en;q=0.9',
    'cache-control': 'no-cache',
    'pragma': 'no-cache',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "YaBrowser";v="24.1", "Yowser";v="2.5"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 YaBrowser/24.1.0.0 Safari/537.36',
}
    # если теперь что-то будет не так, то попробовать requests.Session()
    #подставить ключи если будет надо
    #"X-Api-Key": "-",
    #"X-User-Group": "-",

# Чтение json конфига
with open('config.json') as file:
    config = json.load(file)

mail_login = config['sender login']
mail_password = config['sender password']
pgre_login = config['postgre login']
pgre_password = config['postgre password']
pgre_host = config['postgre host']
pgre_port = config['postgre port']
pgre_db = config['postgre database']
recipients = config['mail recipients']

# Чтение json исключений
with open('exceptions.json', encoding="utf8") as file:
    exceptions_json = json.load(file)

exclude_sellers = exceptions_json['exclude_sellers']
exclude_brands = exceptions_json['exclude_brands']

#Подключение к postgres
conn = psycopg2.connect(
    host = pgre_host,
    port = pgre_port,
    database = pgre_db,
    user = pgre_login,
    password = pgre_password
)

In [19]:
cursor = conn.cursor()

In [4]:
# апи на курсы
curr_api = "https://api.nbrb.by/exrates/rates?periodicity=0"
response = requests.get(curr_api)
curr_resp = response.json()
curr_byn_usd = next((c['Cur_OfficialRate'] for c in curr_resp if c['Cur_Abbreviation'] == 'USD'), None)
curr_byn_eur = next((c['Cur_OfficialRate'] for c in curr_resp if c['Cur_Abbreviation'] == 'EUR'), None)
curr_eur_usd = curr_byn_eur / curr_byn_usd


In [ ]:
115568095, 115254501, 115552403, 115371199

In [1]:
id_value = 115254501

In [5]:
# получения json истории по айди
pr_history_url = f"https://api.av.by/offers/{id_value}/price-history" 
pr_h_response = requests.get(pr_history_url, headers=headers)




In [7]:

# Проверка успешности
if pr_h_response.status_code == 200:
    pr_h_data = pr_h_response.json()  # Получаем джсон
    for i in pr_h_data:
        pr_h_date = i['date']
        pr_h_date = datetime.strptime(pr_h_date, '%Y-%m-%dT%H:%M:%S%z') # Преобразования текстового значения в дату
        pr_h_date = pr_h_date.replace(tzinfo=None) # убираем таймзон
        pr_h_currency = i['currency']
        pr_h_amount = i['amount']
        pr_h_idu = f'{id_value}_{pr_h_date.year}-{pr_h_date.month}-{pr_h_date.day}_{pr_h_date.hour}-{pr_h_date.minute}-{pr_h_date.second}'
        if pr_h_currency == 'byn':
            pr_h_usd_eq = int(round(pr_h_amount / curr_byn_usd, 0))
        elif pr_h_currency == 'eur':
            pr_h_usd_eq = int(round(pr_h_amount * curr_eur_usd, 0))
        else:
            pr_h_usd_eq = pr_h_amount
        # пишем каждую строку а в конце коммит
        print(f'для айди {id_value} - idu {pr_h_idu}. Цена на дату {pr_h_date} составила {pr_h_amount} {pr_h_currency}, в экв. {pr_h_usd_eq} usd')

        

для айди 115254501 - idu 115254501_2025-3-30_14-25-2. Цена на дату 2025-03-30 14:25:02 составила 11000 usd, в экв. 11000 usd
для айди 115254501 - idu 115254501_2025-3-28_17-21-14. Цена на дату 2025-03-28 17:21:14 составила 8200 usd, в экв. 8200 usd
для айди 115254501 - idu 115254501_2025-3-26_16-37-25. Цена на дату 2025-03-26 16:37:25 составила 8000 usd, в экв. 8000 usd
для айди 115254501 - idu 115254501_2025-3-25_7-14-4. Цена на дату 2025-03-25 07:14:04 составила 8050 usd, в экв. 8050 usd
для айди 115254501 - idu 115254501_2025-3-23_13-0-6. Цена на дату 2025-03-23 13:00:06 составила 8100 usd, в экв. 8100 usd
для айди 115254501 - idu 115254501_2025-3-21_17-30-10. Цена на дату 2025-03-21 17:30:10 составила 8200 usd, в экв. 8200 usd


In [21]:
        # пишем
        pr_h_query = """
        INSERT INTO av_price_history (id, id_av, date, amount, currency, usd_eq)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON CONFLICT (id) DO NOTHING
        """
        cursor.execute(pr_h_query, (pr_h_idu, id_value, pr_h_date, pr_h_amount, pr_h_currency, pr_h_usd_eq))
        conn.commit()

In [ ]:
# Функция получения и записи истории изменения цен
def get_price_history (id_value, curr_byn_usd, curr_eur_usd):
    # получения json истории по айди
    pr_history_url = f"https://api.av.by/offers/{id_value}/price-history" 
    pr_h_response = requests.get(pr_history_url, headers=headers)

    # Проверка успешности
    if pr_h_response.status_code == 200:
        pr_h_data = pr_h_response.json()  # Получаем джсон
        for i in pr_h_data:
            pr_h_date = i['date']
            pr_h_date = datetime.strptime(pr_h_date, '%Y-%m-%dT%H:%M:%S%z') # Преобразования текстового значения в дату
            pr_h_date = pr_h_date.replace(tzinfo=None) # убираем таймзон
            pr_h_currency = i['currency']
            pr_h_amount = i['amount']
            pr_h_idu = f'{id_value}_{pr_h_date.year}-{pr_h_date.month}-{pr_h_date.day}_{pr_h_date.hour}-{pr_h_date.minute}-{pr_h_date.second}'
            if pr_h_currency == 'byn':
                pr_h_usd_eq = int(round(pr_h_amount / curr_byn_usd, 0))
            elif pr_h_currency == 'eur':
                pr_h_usd_eq = int(round(pr_h_amount * curr_eur_usd, 0))
            else:
                pr_h_usd_eq = pr_h_amount
            # пишем каждую строку а в конце коммит
            print(f'для айди {id_value} - idu {pr_h_idu}. Цена на дату {pr_h_date} составила {pr_h_amount} {pr_h_currency}, в экв. {pr_h_usd_eq} usd')
            # пишем
            pr_h_query = """
            INSERT INTO av_price_history (id, id_av, date, amount, currency, usd_eq)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT (id) DO NOTHING
            """
            cursor.execute(pr_h_query, (pr_h_idu, id_value, pr_h_date, pr_h_amount, pr_h_currency, pr_h_usd_eq))
            conn.commit()
        return 1
    else:
        print(f"Ошибка {pr_h_response.status_code}: {pr_h_response.text}")
        return 0


In [ ]:
counter = get_price_history(id_value, curr_byn_usd, curr_eur_usd)

для айди 115254501 - idu 115254501_2025-3-30. Цена на дату 2025-03-30 14:25:02 составила 11000 usd, в экв. 11000 usd
для айди 115254501 - idu 115254501_2025-3-28. Цена на дату 2025-03-28 17:21:14 составила 8200 usd, в экв. 8200 usd
для айди 115254501 - idu 115254501_2025-3-26. Цена на дату 2025-03-26 16:37:25 составила 8000 usd, в экв. 8000 usd
для айди 115254501 - idu 115254501_2025-3-25. Цена на дату 2025-03-25 07:14:04 составила 8050 usd, в экв. 8050 usd
для айди 115254501 - idu 115254501_2025-3-23. Цена на дату 2025-03-23 13:00:06 составила 8100 usd, в экв. 8100 usd
для айди 115254501 - idu 115254501_2025-3-21. Цена на дату 2025-03-21 17:30:10 составила 8200 usd, в экв. 8200 usd


In [ ]:

    return 1
else:
    print(f"Ошибка {phone_response.status_code}: {phone_response.text}")
    return 0

In [8]:
conn.close()